In [1]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
import os
from openai import AzureOpenAI

In [2]:
dataset_path = '../dataset/'
train_test_path = dataset_path + 'train_test/'

azure_credentials = {
                        "api_key": "",
                        "api_version": "2024-02-01",
                        "azure_endpoint": "",
                    }

client = AzureOpenAI(
    api_key= azure_credentials['api_key'],  
    api_version= azure_credentials['api_version'],
    azure_endpoint = azure_credentials['azure_endpoint']
    )

In [3]:
train_data = pd.read_csv(train_test_path + 'train.csv')
test_data =  pd.read_csv(train_test_path + 'test.csv')

## Create Files

**GPT's System Prompt:** You are a linguistic researcher rating hate speech intensity in sentences on a scale from 1 to 10, where 1 is minimal and 10 is extreme. This helps create a dataset for researchers to filter harmful content effectively.

In [4]:
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)
system_prompt = """
You are a linguistic researcher specializing in evaluating hate speech intensity in sentences. Your task is to rate the intensity of hate speech on a scale from 1 to 10, where 1 represents minimal intensity and 10 represents extreme intensity. This evaluation helps in creating a dataset for researchers to effectively filter harmful content. Please use the following guidelines for rating the intensity:

1. Score [1 − 3]: The sample uses dark humor or implicit hateful terms.
2. Score [4 − 5]: The sample mainly consists of offensive hashtags, or most hateful phrases are in the form of offensive hashtags.
3. Score [6 − 7]: The sample is mainly composed of sexist/racist terms or portrays a sense of gender/racial superiority by the person sharing the sample.
4. Score [8 − 10]: The sample promotes hate crime and calls for violence against the individual/group.

Your ratings should reflect the content and context of each sample according to these guidelines.
"""

# Function to convert DataFrame to JSONL format for GPT-3.5 Turbo
def convert_to_jsonl(data, file_path):
    jsonl_data = []
    for index, row in data.iterrows():
        jsonl_data.append({
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": row['sentence']},
                {"role": "assistant", "content": str(row['intensity'])}
            ]
        })

    with open(file_path, 'w') as outfile:
        for entry in jsonl_data:
            json.dump(entry, outfile)
            outfile.write('\n')

convert_to_jsonl(train_data, train_test_path + '(i)gpt/gpt_intensity_train.jsonl')
convert_to_jsonl(val_data, train_test_path + '(i)gpt/gpt_intensity_val.jsonl')

## Upload Files

In [5]:
training_file_name = train_test_path + '(i)gpt/gpt_intensity_train.jsonl'
validation_file_name = train_test_path + '(i)gpt/gpt_intensity_val.jsonl'

# def upload_train_test(client, training_file_name, validation_file_name)
# Upload the training and validation dataset files to Azure OpenAI with the SDK.
training_response = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response.id

validation_response = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-6a553ea0479347a8a36fc0f4eface8cc
Validation file ID: file-ade6f0a090f145ba8c819672caefa690


## Finetune GPT

In [6]:
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-35-turbo-0125",
    # hypperparameters={}
)

job_id = response.id

# You can use the job ID to monitor the status of the fine-tuning job.
# The fine-tuning job will take some time to start and complete.

print("Job ID:", response.id)
print("Status:", response.status)
print(response.model_dump_json(indent=2))

Job ID: ftjob-4859bb787879422daf071793c0912f17
Status: pending
{
  "id": "ftjob-4859bb787879422daf071793c0912f17",
  "created_at": 1720255215,
  "error": null,
  "fine_tuned_model": null,
  "finished_at": null,
  "hyperparameters": {
    "n_epochs": -1,
    "batch_size": -1,
    "learning_rate_multiplier": 1
  },
  "model": "gpt-35-turbo-0125",
  "object": "fine_tuning.job",
  "organization_id": null,
  "result_files": null,
  "seed": null,
  "status": "pending",
  "trained_tokens": null,
  "training_file": "file-6a553ea0479347a8a36fc0f4eface8cc",
  "validation_file": "file-ade6f0a090f145ba8c819672caefa690",
  "estimated_finish": null,
  "integrations": null
}


## Manually create a deployment

In [6]:
def get_reduced_intensity(sentence, deployment_name, system_prompt):
    try:
        response = client.chat.completions.create(
        temperature = 0.5,
        model= deployment_name,
        messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"'{sentence}'"}
            ],
        max_tokens=1000,  # Increase this value to accommodate the full JSON response
        )
        print(sentence, response.choices[0].message.content)
        return response.choices[0].message.content

    ## gpt restrictions - cannot process hate speech
    except Exception as e:
        return 11.0

In [8]:
train_data = pd.read_csv(train_test_path + 'train.csv')
test_data =  pd.read_csv(train_test_path + 'test.csv')

# Apply the get_reduced_intensity function to the "Sentence" column
deployment_name = "intensity-update"
train_data['gpt_35_intensity'] = train_data['sentence'].apply(lambda x: get_reduced_intensity(x, deployment_name, system_prompt))
train_data.to_csv('../dataset/result/train_gpt_35_updated.csv', index=False)
train_data

my coworkers saw an ugly side of me today  thanks to everyone who listened to me foul woman today tho ily 2
@user @user Bitch stfu ! Im just tired of you thinking you know me because of what that bum ass nigga tells you when he needs some pussy or a place to sleep 8
Just when things are going good women will mess it up. They're not happy unless there's turmoil 5
Alcohol doesn't give you an excuse to be a hustler 6
@user We need our ARMY down there to #StopTheInvasion ! ! ! WHERE ARE THEY that is More Important than defending our national border against this constant onslaught of hostile foreign invaders ? ! ! ? ! ? ? ! 7
By their words and deeds , they declare themselves to be staunchly against people . 6
Its time to end Obama's DACA and don't reward Illegal Immigration U.S Youths need Jobs 6
@user @user @user prime raped bitch 8
the flag of the maga fags in all it glory it comes with african american red shoes too 7
as fully vested members of the same grisly club you can always count 

all of your videos you stupid video game criticizing woman stop talking about things you know nothing about 2
We aren't allowed to build churches in their countries, why should they be allowed to build Mosques in ours? 7
i do not know it does not matter anyway i only have some money but i would jump into a sewer before i swam in a pool or anything with a nigger i mean wtf are ppl thinking 7
@user Please insist on wall funding now ! Please do not let the speaker of the house and senate leader talk you into waiting to after the election for the funding . after the election you will be dealing with a lame duck congress . #BuildTheWall # B 7
Why would u play with my pure heart ? 5
Crimes committed by ILLEGAL immigrants should be public information everywhere like in Texas . More illegal aliens were arrested for homicide in the past 7 years in a single state than deaths by mass shooting in the other 49 states combined ! #SendThemBack #BuildTheWall 7
Hoe you watching I suggest you keep watch

U.S. Marshals: Illegal immigrants wanted for murder in Mexico arrested in SC 5
lol what a stupid person 2
stunting on all yall hoes that was flexing on me 6
I'm only going out with doctors and lawyers from now on. At least if they go nuts I know they have somethin to lose unlike these woman workin retail 6
Media has sold out to Soros, etc, and selling USA Public down the river for power and money, we are nothing to them Ask them to leave the nation. enough already. 5
youre not going to tell me to say happy birthday to someone that called me out within the span of 10 minutes 6
i am totally sure she dindunuffin grandpa i mean in this day and age it literally tyrannical to ask that your woman show a little bit of public decency not be with random despicable people of african origin behind your back and care if her child is actually yours all the signs were there though 2
relationship be so good a women be slow stroking and crying 2
Germany : Mass migration of un-assimilatable illegal alie

Ask any liberal how many illegal immigrants they've let stay in their house. The answer probably won't surprise you. Ask them to leave the nation. 6
Gels getting overly emotional about things that don't matter ? Color me shocked ! #WomenSuck 7
i did not own any slaves my ancestors never owned a slave i work my ass off for every dollar i get so shove this crap up your bigoted thieving nigger ass ms never was a slave 7
@user the majority of abusers are of anglo descent not many foul african american are abusers it seem to be a desire of most anglos to as they say break in a child or tame them to comply white kids are more likely to be abused by someone they know 3
@user I have never seen such ignorant, illiterate people in my life. Bring in Natl Guard and disperse 5
@user It's kind of funny , really . Were the space aliens illegal ? Now we need a wall & a roof ! I think this country has gone totally nuts . God Bless @user for his strength & courage . #BuildThatWall and if he blocks the d

Democratics pls call your representatives and demand Kavanaugh be confirmed, esp if up for re-election.Build the wall 6
Now now lets not discriminate between muslims and jews , I say get them all back to the middle east and turn the whole sand pit to radioactive glass .  8
let masha and the dreaded women send that one to foul african american hebben 2
Well, at least these Africans got the fact that Jews are corrupt right ; though I suspect Whites and Jews appear the same to them. 7
harry kane is a fucking fanny if he was foriegn the commentators would be going on about how bad diving is in the game absolute wankers 6
In addition to possibly losing a few million holidaymakers because of problems with all male , violent African " refugees " Spain could also lose fortunes from ex pats who would move back to the UK and others who would not buy property in Spain . Why does Spain take migrants ?  8
I love it when married bitches start posting inspirational memes about having a strong relatio

...

yes he respects women and is totally not a repulsive fou skin colored super beta who has to badger women until he is satisfied mpc status updates 2
Dumbest user on twitter so far! Equates  not letting someone into the country illegally, to gassing  Ask them to leave the nation. Ask them to leave the nation. 6
@user We need our ARMY down there to #StopTheInvasion ! ! ! WHERE ARE THEY that is More Important than defending our national border against this constant onslaught of hostile foreign invaders ? ! ! ? ! ? ? ! 8
@user whooooo ? Chelsey ? Fuck her lol . She juss a bitch 7
well when i was in the air force they thought i was a dyke plumber bitch so there ya go opinions are like assholes 7
god dam these nigger biches ugly ass fuck 9
I would not be surprised if this woman was specifically targeted by local Jews from her town for this abominable treatment . 8
Some muslims misinterpret that Islamic ideology profess hate and violence and oppression of women and people with a different sexu

@user Fa Real Desperate Ass Hoe­ A Fat Bitch And A Real Skinny Bitch­ How 8
@user Whatever ! Fuck all those assholes . They're niggers , they all look alike to me .  7
that because those tv students are jewish actors as is sheriff israel and the shooter himself the foul spanish speaking women is just a awful spanish speaking person 2
you foul african american gay a war is coming you and your low level fellows will not survive it 2
Teach our men to be men again and defend our countries and women folk against the other people . 6
Women only vote for question 6 cuz their 'best friend' is a homo or Chad at the salon told them to. 7
guy : " i want a girl who's smart . but i also want a girl who is a total and complete slut whore . " me : " how bout a smart slutwhore ? "  7
Once a meathead always a meathead. Illegal immigrants is not immigration. 7
youre a women dont speak to me again u fake woman 2
It's kind of funny, really. Were the space immigrants illegal? Now we need a wall & a roof! I

sorry about the nigger comment but even blacks do not like niggers 7
Jervea ass bout to really come up missing stupid hoe got me sick 7
why did i as a teen look up to the basic white bitches amongst me at school 6
I do not blame you , Ottawa like all of our cities is a multicultural hell .  8
since the day some pak muslims set foot in the uk they have been harming white children young white lads defended their females and were called awful pakistani bashers and nazis unfairly i want pak community sent back some bad ones from them theyre all in it encouraged by family 7
my hate for @user is getting stronger and stronger every time i watch her . #MKR 4
the muzzie looks miserable and ugly 7
@user look at this african origin person right here 2
maybe if you women weren't so indecisive you'd understand 3
Well The Girls might pull this off . Pardon the pun ! 6
Whether you accept it or not, people are concerned with Muslim criminal gangs and the spread of hate. 7
@user lmao he shoulda said I 

Couple Befriend Negroid , Negroid Tries To Burn Down Couples Home - NNN Reporters Newsroom Forum Moblie Couple Says Neighbor Tried to Set Their Apartment on Fire Reported by : Irika Sargent Last Update : 4/25 9 : 01 pm Moblie Couple Says Neighbor Tried to Set Their Apartment on Fire | Local15tv.com , Mobile & Pensacola News , Entertainment , Videos , Business Search and Shopping This is what happens when you make friends with spearchuckers .  7
@user @user Fucking bitch ! Shes doing that since 2 years when she give money to STUDENTS , STUPID HOE ! Now laugh in silence and poverty ! U want attention and like bcse its trendy ton talk shit about her ? ?  8
You could say that nowadays only 5-10% of Muslims are extremists, but remember that in 1940 only 7% germans were Nazis. 8
can you all foul african americans stop asking if i am sick that just how i look 2
queer eye is my shit a bitch is addicted 7
Or try yelling * * * * the nigs ya ' ll get suspended , jumped and hopefully some new wn f

,sentence,intensity,gpt_35_intensity
0,my coworkers saw an ugly side of me today tha...,2,2
1,@user @user Bitch stfu ! Im just tired of you ...,9,8
2,Just when things are going good women will mes...,4,5
3,Alcohol doesn't give you an excuse to be a hus...,6,6
4,@user We need our ARMY down there to #StopTheI...,8,7
...,...,...,...
4838,Some vulnerable sections are harmed by some Mu...,5,5
4839,why can not i hold all this projection i could...,2,7
4840,done wit yo awful african american i just want...,2,2
4841,Hazel e is a joke. Girl you bashing dark skin ...,5,6


In [9]:
deployment_name = "intensity-update"
test_data['gpt_35_intensity'] = test_data['sentence'].apply(lambda x: get_reduced_intensity(x, deployment_name, system_prompt))
test_data.to_csv('../dataset/result/test_data_gpt_35_updated.csv', index=False)
test_data

people think illegal immigration is just about jobs. and . . . no one talks about the fraud committed or money sucked out by from social services 5
and if yo bitch fast prolly cuz she out here runnin 8
lets hope so gas the lot including the nigger 7
It is safer not to share a meal with some Muslim, because you catch a disease 5
who cares about the kneeling foul african american football league not me their hero and ringleader unemployed jerk and his foul muslim girlfriend are stupid dumb people with a low sum iq 2
but that you can not even articulate bad jews what is yes a foul jew is anyone who signals against myself or any other alt right person who only exposes jews and my posts judaism v the world among other posts have brought you to my timeline despicable jew 2
You could say that nowadays only some Muslims are extremists, but remember that only some people cause issues. 5
@user @user Michelle Obama is a woman that every woman should aspire to be where Flotus is a skank that no on

I do not really understand why I should listen to some Muslim women. They are not well educated. 7
@user Oh @user you do something new every night to make yourself look even more like a rude person! 3
i know the awful white woman is going to jail cuz she a woman but still lmaoo good riddance awful woman 2
The same Max Mosley who published a by-election leaflet saying : " Coloured immigration threatens your children's health , " @url 7
nothing regarding any nigger at school or in the work sector is based on merit you fuckers get special treatment all standards are lowered for you what are you writing your thesis on the colonel special herbs and spices 7
look in the mirror, complete trash you are you make women yourself and the great saint 2
Everything will be better if some muslims did not misinterpret Islam and do some things. Some of them spread distrubance in the society 4
The only purpose of Islam is to dominate and destroy us! Being multicultural has nothing to do with it. 8
clearl

@user @user is a scheming fraud who fronts a country group 2
@user i am bit confused coz chinese ppl can not access twitter thn how this ching chong using it i think he pakistani 7
In the swimming pool there are baths only for Muslims, I cannot enter. If I, a white man, asked for something like this nobody would allow me. Muslims are getting priveleges. 7
assuming we are on the same side edgy morons like yourself are the enemy have fun alienating people who do not worship your kike on a stick 7
@user Go pray in a church or something, no one cares about your religious thing. 3
forever lean stripper pole bitch this nigger purple sprite 7
This is what the streets of the #uk will look like in 10 more years #Brexitnow #stopimmigration 7
according to that piranha mouth despicable lesbian i am a level foul south east asian 2
fuck them white kids 6
The liberal government that rules our country persists in defending crimes done by some Muslims because they want to spread modern values. They wea

Illegal Aliens Who Murdered Man In North Carolina Captured via @user #BuildTheWall #KeepAmericansSafe #EndDACA #DeportThemAll #DeathPenalty 8
You people are all welcome to this place if you can perform the ' small ' task of riding this place of it 's people ! 6
All muslim women are stupid: we should not be influenced by them. 7
Wow . Wiped that bitchy smile right of your face @user #MKR #Fail 7
kevin friend is a total joke of a referee munars 6
welp gave the negro too much credit because he currently ruining my bath by coming in here taking a shit 8
No your not ! You are scroungers / soldiers coming to create a caliphate or create chaos . Go home , look after your girlfriends / wives / kids . All fighting age . The last thing you are , are refugees ! Bugger off ! Bye , bey , cheers ... @user  7
@user @user You mean illegal aliens that should have applied for visas like real immigrants do . Boo hoo 7
i love your pro white jack o lantern fuck that nigger and fuck the national african ame

So many woman texted Stephen when we got Lionel 'our cats look the same!' Women 95% of house cats in the US are domestics, stay in your lane 5
@user wow ! ! ! Are you comin back up here ? they some bitch niggas 7
You might say, Don't worry, only some Muslims are bad. Remember, earlier only few people caused issues. 4
just another dumb nigger 7
the labour party has mps in prison for paedophilia mps and councillors on the sex registry million white kids raped by paki muslims all labour voters covered up by labour councils mps and a pm 8
I think that Ramadam will not prevent some Muslims who misinterpret Islam from committing crimes 4
When y'all see that bitch that's been talkin shit , but you refrain cause you've been tryin to grow as a person @url 7
he rattled a stupid jew 2
Go on talking to yourself, no one will listen. Islam does not belong within western society. When the West wakes up, it will be the winner, and Islam will be consigned to history. 7
refugees in white homelands are w

...

I hate when girls use guys . We're not toys damn it ! #WomenSuck 8
you said if someone calls you hot 1 more time , you are gonna do what they want don't back off now 5
ENOUGH w / PHONY LIB argumentsvoter IDs don't workWalls don't workIllegals less crimeimmigration system brokeillegal kids are dreamers ; OUR kids are notmore gun laws neededblacks can't be racist #POTUS #KAG #MAGA #NEWS #Trump #Walkaway #VoteRed #BuildTheWall #NoDACA #NRA 7
Well Kat... that's what happens when you're a hustler 6
Trust me The girl ting works for him, just stay in your lane 6
@user needs to send in the Military to the southern border stop this invasion of America from the 3rd world . #MAGA #BuildTheWall #NoDACA #NoAmnesty #DeportAllOfThem 7
wishing the happiest birthday to my favorite drug @user love u foul women 2
@user @user cuz ppl r not taught the full history usage it lessens understanding of why it so offensive a history literature teacher sayin nigger coon spick wetback towelhead chink when examinin

Muslims are two-faced, have you ever seen a Muslim protest against Isis or Saudi human right abuses? 8
i am not saying the n word because i am racist i am saying it to take a stand against pc and fight for freedom of speech white dude comedians 7
i disrespected the holocaust museum and did not get caught gr baby joaquen phoenix in the joker 6
They instead decide by convenience , low risk , and unenlightened self-interest . 5
You pick the guy thats a total douche to you over the one whose good to you . Got it . Makes perfect sense ! #WomenSuck 8
@user @user no one inayah a mentally weak awful gay rather have porridge that the reason u numpty you are big gay 2
people undergo the level of degradation that lee ermey dishes out just to be on tv they do not get called faggot or nigger so i guess that makes it ok 7
I would just like to express how sick and tired I am of bi-racial half-breeds kids at school boasting about their ` ` unique heritages ' ' . 8
Jews have lived in England for centur

@user @user @user coming to the rescue of some retard licking dirty skidmarks on the internet ... you guys are fucktards 8
One of the biggest reason I never wanted to come out to my friends was bc I was afraid theyll think I have crush on them when Im just being a decent human being so I call them stupid bitch and a fucking hoe everytime I see them 8
@user ah man i fucking hate you so much 4
Immigrants Using Nearly 60 Percent More Food Stamps than Americans . Taxpayers pay Approx . $ 48,000 . 00 per Illegal Alien per Year ! @url 8
@user bernardo is just a white honky 6
twomad was having too much fun in overwatch he said live on stream hola my awful african americans 2
the first white president since were going into skin color now obama was the first nigger president lets return to the good old times 8
DREAMER' Sodomized Multiple Boys, Between The Ages 7 and 9 via is too good for this dispicable Ask them to leave the nation. What about the dreams this POS Stole from these children?I hop

Justtalk bad, woman because you dont get the attention you want or the singular self-importance. 2
@user @user red i fucking hate you for retweeting this 7
Just see the number of immigrants coming in!! God Bless Trump!Ask them to leave the nation. 6
Is it tv editing or is @user a naturally impolite person 2
Move on and stop going back to the same asshole ! Omg you dumb bitch ! #WomenSuck 7
Fast-Food Workers Union Demands Restaurant to Allow Employees to Wear ' Abolish ICE ' Pins | Breitbart #BuildTheWALL #AmericaFirst #NoDACA #NoDEMS #GreatAwakening #MAGA #WalkAway #ThesePeopleAreSick #ICE 7
Cheerleaders make me fucking sick ! It's not a sport you dumb skanks so go back to caking on your makeup . #WomenSuck 7
dont come to me as a woman .... Cuz yo came at me single 5
@user @user another pakistani jihadi hiding behind a fake name 8
Meanwhile those nonwhites continue to breed even as their nonwhite societies continue to inflict their harsh uncivilized nonwhite culture on us . 8
how about

,sentence,intensity,gpt_35_intensity
0,people think illegal immigration is just about...,4,5
1,and if yo bitch fast prolly cuz she out here r...,6,8
2,lets hope so gas the lot including the nigger,7,7
3,It is safer not to share a meal with some Musl...,4,5
4,who cares about the kneeling foul african amer...,2,2
...,...,...,...
1206,i stand with brianna brochu because being forc...,2,2
1207,@user kill yourself o,6,7
1208,now you see why god told his people to kill qu...,6,9
1209,i think the one she replaced was actually a co...,2,2


## Calculating RMSE, Pearson and Cosine

In [35]:
import numpy as np
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity

def metric (compared_against):
    # Convert columns to numeric types
    test_data['intensity'] = pd.to_numeric(test_data['intensity'])
    test_data[compared_against] = pd.to_numeric(test_data[compared_against])
    
    # Define the data
    intensity = test_data['intensity'].values
    predicted_intensity = test_data[compared_against].values
    
    # Compute RMSE
    rmse = np.sqrt(mean_squared_error(intensity, predicted_intensity))
    
    # Compute Pearson correlation
    pearson_corr, _ = pearsonr(intensity, predicted_intensity)
    
    # Compute Cosine similarity
    cosine_sim = cosine_similarity([intensity], [predicted_intensity])[0][0]
    
    return (rmse, pearson_corr, cosine_sim)

### GPT 4o

In [10]:
deployment_name = "intensity-with-context"
test_data['gpt_4o_intensity'] = test_data['sentence'].apply(lambda x: get_reduced_intensity(x, deployment_name, system_prompt))

In [11]:
print(metric('gpt_4o_intensity'))
print(metric('predicted_intensity'))

NameError: name 'metric' is not defined

In [40]:
test_data.to_csv('../dataset/result/test_data_gpt_35_4o.csv', index=False)